In [ ]:
!nvidia-smi

Sun Aug 14 13:35:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!rm -rf XLNet

In [ ]:
!cp -r /content/drive/MyDrive/Euphemism22 .

cp: cannot stat '/content/drive/MyDrive/Euphemism22': No such file or directory


In [ ]:
!cp -r /content/drive/MyDrive/Euphemism22_code .

cp: cannot stat '/content/drive/MyDrive/Euphemism22_code': No such file or directory


In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 6.0 MB/s 


In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.6 MB 7.4 MB/s 
     |████████████████████████████████| 101 kB 14.1 MB/s 
     |████████████████████████████████| 596 kB 75.5 MB/s 


In [ ]:
import torch

In [ ]:
import io
import os
import torch
import random
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
# from ml_things import plot_dict, plot_confusion_matrix, fix_text
from sklearn.metrics import classification_report, accuracy_score, f1_score
from transformers import (set_seed,
                          TrainingArguments,
                          Trainer,
                          # XLNetConfig,
                          AutoConfig,
                          AutoTokenizer,
                          AdamW, 
                          Adafactor,
                          get_linear_schedule_with_warmup,
                          get_cosine_with_hard_restarts_schedule_with_warmup,
                          AutoModelForSequenceClassification)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [ ]:
# Set seed for reproducibility.
SEED = 123
set_seed(SEED)

# Number of training epochs (authors on fine-tuning Bert recommend between 2 and 4).
epochs = 40

# Number of batches - depending on the max sequence length and GPU memory.
# For 512 sequence length batch of 10 works without cuda memory issues.
# For small sequence length can try batch of 32 or higher.
batch_size = 16

# Pad or truncate text sequences to a specific length
# if `None` it will use maximum sequence of word piece tokens allowed by model.
max_length = 150 # 770 for concat

# Look for gpu to use. Will use `cpu` by default if no gpu found.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Name of transformers model - will use already pretrained model.
# Path of transformer model - will load your own model from local disk.
model_name_or_path = 'cardiffnlp/twitter-roberta-base-sentiment-latest'

# Dictionary of labels and their id - this will be used to convert.
# String labels to number ids.
# with open('data_json/label_dict.json','r') as f:
#   labels_ids = json.load(f)

# How many labels are we using in training.
# This is used to decide size of classification head.
# n_labels = len(labels_ids)

# learning rate for the optimizer
lr_param = 2.5e-5

In [ ]:
import sys
sys.path.append('Euphemism22_code')

In [ ]:
from Euphemism22Dataset import Euphemism22Dataset
from Euphemism22ClassificationCollator import Euphemism22ClassificationCollator

ModuleNotFoundError: ignored

In [ ]:
def train(dataloader, valid_dataloader, optimizer_, scheduler_, device_, eval_every=400):
    """
    Train pytorch model on a single pass through the data loader.

    It will use the global variable `model` which is the transformer model 
    loaded on `_device` that we want to train on.

    This function is built with reusability in mind: it can be used as is as long
      as the `dataloader` outputs a batch in dictionary format that can be passed 
      straight into the model - `model(**batch)`.

    Arguments:

        dataloader (:obj:`torch.utils.data.dataloader.DataLoader`):
            Parsed data into batches of tensors.

        optimizer_ (:obj:`transformers.optimization.AdamW`):
            Optimizer used for training.

        scheduler_ (:obj:`torch.optim.lr_scheduler.LambdaLR`):
            PyTorch scheduler.

        device_ (:obj:`torch.device`):
            Device used to load tensors before feeding to model.

    Returns:

        :obj:`List[List[int], List[int], float]`: List of [True Labels, Predicted
          Labels, Train Average Loss].
    """

    # Use global variable for model.
    global model
    global best_acc
    global SEED

    # Tracking variables.
    predictions_labels = []
    true_labels = []
    # Total loss for this epoch.
    total_loss = 0

    # Put the model into training mode.
    model.train()

    n_iter = 0

    # For each batch of training data...
    for batch in tqdm(dataloader, total=len(dataloader)):
        n_iter += 1

        # Add original labels - use later for evaluation.
        true_labels += batch['labels'].numpy().flatten().tolist()
        
        # move batch to device
        batch = {k:v.type(torch.long).to(device_) for k,v in batch.items()}
        
        # Always clear any previously calculated gradients before performing a
        # backward pass.
        model.zero_grad()

        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this a bert model function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(**batch)

        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple along with the logits. We will use logits
        # later to calculate training accuracy.
        loss, logits = outputs[:2]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()

        # Convert these logits to list of predicted labels values.
        predictions_labels += logits.argmax(axis=-1).flatten().tolist()

        if n_iter % eval_every == 0:
            # Get prediction form model on validation data. 
            print('Validation on batches...')
            valid_labels, valid_predict, val_loss = validation(valid_dataloader, device)
            val_acc = accuracy_score(valid_labels, valid_predict)
            val_f1 = f1_score(valid_labels, valid_predict)
            print("step: %d   val_loss: %.5f - valid_acc: %.5f"%(n_iter, val_loss, val_acc))

            if val_acc >= best_acc:
                model.save_pretrained("XLNet/lr_{}_best.pt".format(lr_param))
                best_acc = val_acc
                print("model saved   best_valid_acc: %.5f"%(val_acc))

    # Calculate the average loss over the training data.
    avg_epoch_loss = total_loss / len(dataloader)
    # SEED += random.randint(3, 12)
    # set_seed(SEED)
    # Return all true labels and prediction for future evaluations.
    return true_labels, predictions_labels, avg_epoch_loss

In [ ]:
# Get model configuration.
print('Loading configuraiton...')
# model_config = XLNetConfig.from_pretrained(pretrained_model_name_or_path=model_name_or_path, num_labels=2)
model_config = AutoConfig.from_pretrained(model_name_or_path)

# Get model's tokenizer.
print('Loading tokenizer...')
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name_or_path)
# default to left padding
tokenizer.padding_side = "left"
# Define PAD Token = EOS Token = 50256
tokenizer.pad_token = tokenizer.eos_token


# Get the actual model.
print('Loading model...')
model = AutoModelForSequenceClassification.from_pretrained(pretrained_model_name_or_path=model_name_or_path)
model.classifier.out_proj = torch.nn.Linear(in_features=768, out_features=2, bias=True)
model.num_labels = 2
model_config.num_labels = 2
# resize model embedding to match new tokenizer
model.resize_token_embeddings(len(tokenizer))

# fix model padding token id
model.config.pad_token_id = model.config.eos_token_id

# Load model to defined device.
model.to(device)
print('Model loaded to `%s`'%device)

In [ ]:
def validation(dataloader, device_):
    """Validation function to evaluate model performance on a 
    separate set of data.

    This function will return the true and predicted labels so we can use later
    to evaluate the model's performance.

    This function is built with reusability in mind: it can be used as is as long
      as the `dataloader` outputs a batch in dictionary format that can be passed 
      straight into the model - `model(**batch)`.

    Arguments:

      dataloader (:obj:`torch.utils.data.dataloader.DataLoader`):
            Parsed data into batches of tensors.

      device_ (:obj:`torch.device`):
            Device used to load tensors before feeding to model.

    Returns:
      
      :obj:`List[List[int], List[int], float]`: List of [True Labels, Predicted
          Labels, Train Average Loss]
    """

    # Use global variable for model.
    global model

    # Tracking variables
    predictions_labels = []
    true_labels = []
    #total loss for this epoch.
    total_loss = 0

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Evaluate data for one epoch
    for batch in tqdm(dataloader, total=len(dataloader)):

        # add original labels
        true_labels += batch['labels'].numpy().flatten().tolist()

        # move batch to device
        batch = {k:v.type(torch.long).to(device_) for k,v in batch.items()}

        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(**batch)

            # The call to `model` always returns a tuple, so we need to pull the 
            # loss value out of the tuple along with the logits. We will use logits
            # later to to calculate training accuracy.
            loss, logits = outputs[:2]
            
            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()

            # Accumulate the training loss over all of the batches so that we can
            # calculate the average loss at the end. `loss` is a Tensor containing a
            # single value; the `.item()` function just returns the Python value 
            # from the tensor.
            total_loss += loss.item()
            
            # get predicitons to list
            predict_content = logits.argmax(axis=-1).flatten().tolist()

            # update list
            predictions_labels += predict_content

    # Calculate the average loss over the training data.
    avg_epoch_loss = total_loss / len(dataloader)

    # Return all true labels and prediciton for future evaluations.
    return true_labels, predictions_labels, avg_epoch_loss

In [ ]:
# Create data collator to encode text and labels into numbers.
kpa_classification_collator = Euphemism22ClassificationCollator(use_tokenizer=tokenizer, 
                                                          max_sequence_len=max_length)


print('Dealing with Train...')
# Create pytorch dataset.
train_dataset = Euphemism22Dataset(path='Euphemism22',
                                  split='alltrain')
print('Created `train_dataset` with %d examples!'%len(train_dataset))

# Move pytorch dataset into dataloader.
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=kpa_classification_collator)
print('Created `train_dataloader` with %d batches!'%len(train_dataloader))

print()

print('Dealing with Validation...')
# Create pytorch dataset.
valid_dataset =  Euphemism22Dataset(path='Euphemism22',
                                   split='val')
print('Created `valid_dataset` with %d examples!'%len(valid_dataset))

# Move pytorch dataset into dataloader.
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, collate_fn=kpa_classification_collator)
print('Created `eval_dataloader` with %d batches!'%len(valid_dataloader))

In [ ]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = lr_param, # default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # default is 1e-8.
                  )
# optimizer = Adafactor(model.parameters(),
#                   lr = lr_param, # default is 5e-5, our notebook had 2e-5
#                   relative_step = False
#                   )

# Total number of training steps is number of batches * number of epochs.
# `train_dataloader` contains batched data so `len(train_dataloader)` gives 
# us the number of batches.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)
scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
    optimizer, 
     num_warmup_steps = 0, # Default value in run_glue.py
    num_training_steps = total_steps,
    num_cycles = 5
)

# scheduler = None
# Store the average loss after each epoch so we can plot them.
all_loss = {'train_loss':[], 'val_loss':[]}
all_acc = {'train_acc':[], 'val_acc':[]}
all_f1 = {'train_f1':[], 'val_f1':[]}


In [ ]:
# Loop through each epoch.
best_f1 = 0.0
print('Epoch')
for epoch in tqdm(range(epochs)):
    print()
    print('Training {} on batches...'.format(epoch))
    # Perform one full pass over the training set.
    train_labels, train_predict, train_loss = train(train_dataloader, valid_dataloader, optimizer, scheduler, device)
    train_acc = accuracy_score(train_labels, train_predict)
    train_f1 = f1_score(train_labels, train_predict)

    # Get prediction form model on validation data. 
    print('Validation on batches...')
    valid_labels, valid_predict, val_loss = validation(valid_dataloader, device)
    val_acc = accuracy_score(valid_labels, valid_predict)
    val_f1 = f1_score(valid_labels, valid_predict)

    # Print loss and accuracy values to see how training evolves.
    print("epoch: %d  train_loss: %.5f - val_loss: %.5f - train_acc: %.5f - valid_acc: %.5f"%(epoch+1, train_loss, val_loss, train_acc, val_acc))
    print("epoch: %d  train_f1: %.5f - val_f1: %.5f -"%(epoch+1, train_f1, val_f1))
    print()

    # Store the loss value for plotting the learning curve.
    all_loss['train_loss'].append(train_loss)
    all_loss['val_loss'].append(val_loss)
    all_acc['train_acc'].append(train_acc)
    all_acc['val_acc'].append(val_acc)
    all_f1['train_f1'].append(train_f1)
    all_f1['val_f1'].append(val_f1)
    if train_f1 >= best_f1 or (epoch + 1) % 5 == 0:
        model.save_pretrained("XLNet/lr_{}_best_ep{}.pt".format(lr_param,epoch))
        best_f1 = train_f1
        print("model saved   best_valid_f1: %.5f"%(train_f1))


Epoch


  0%|          | 0/40 [00:00<?, ?it/s]


Training 0 on batches...


  0%|          | 0/99 [00:00<?, ?it/s]

Validation on batches...


...

epoch: 39  train_loss: 0.00519 - val_loss: 0.00002 - train_acc: 0.99873 - valid_acc: 1.00000
epoch: 39  train_f1: 0.99910 - val_f1: 1.00000 -


Training 39 on batches...


  0%|          | 0/99 [00:00<?, ?it/s]

Validation on batches...


  0%|          | 0/10 [00:00<?, ?it/s]

epoch: 40  train_loss: 0.01478 - val_loss: 0.00002 - train_acc: 0.99809 - valid_acc: 1.00000
epoch: 40  train_f1: 0.99864 - val_f1: 1.00000 -

model saved   best_valid_f1: 0.99864


In [ ]:
# !cp -r XLNet/lr_2.5e-05_best_ep30.pt /content/drive/MyDrive/Euphemism22-roberta-base-sentiment-latest-cosine/

In [ ]:
# !mkdir -p /content/drive/MyDrive/Euphemism22-roberta-base-sentiment-latest-cosine

In [ ]:
print('Loading model...')
test_epoch = 37
pretrained_path = "XLNet/lr_{}_best_ep{}.pt".format(lr_param, test_epoch)
model = AutoModelForSequenceClassification.from_pretrained(pretrained_model_name_or_path=pretrained_path, num_labels= 2, ignore_mismatched_sizes=True)

# resize model embedding to match new tokenizer
# model.resize_token_embeddings(len(tokenizer))

# fix model padding token id
model.config.pad_token_id = model.config.eos_token_id

# Load model to defined device.
model.to(device);

Loading model...


In [ ]:
model_config.num_labels

2

In [ ]:
# # Name of transformers model - will use already pretrained model.
# # Path of transformer model - will load your own model from local disk.
# model_name_or_path = 'cardiffnlp/twitter-roberta-base-offensive'

# # Get model configuration.
# print('Loading configuraiton...')
# # model_config = XLNetConfig.from_pretrained(pretrained_model_name_or_path=model_name_or_path, num_labels=2)

# # Get model's tokenizer.
# print('Loading tokenizer...')
# tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name_or_path)
# # default to left padding
# tokenizer.padding_side = "left"
# # Define PAD Token = EOS Token = 50256
# tokenizer.pad_token = tokenizer.eos_token

# # Get the actual model.
# print('Loading model...')
# pretrained_path = '/content/drive/MyDrive/Euphemism22-XLNet-base-best-v2/lr_{}_best.pt'.format(lr_param)
# model = AutoModelForSequenceClassification.from_pretrained(pretrained_model_name_or_path=pretrained_path)

# # resize model embedding to match new tokenizer
# model.resize_token_embeddings(len(tokenizer))

# # fix model padding token id
# model.config.pad_token_id = model.config.eos_token_id

# # Load model to defined device.
# model.to(device)
# print('Model loaded to `%s`'%device)

In [ ]:
model.eval();

In [ ]:
from Euphemism22TestDataset import Euphemism22TestDataset
from Euphemism22ClassificationTestCollator import Euphemism22ClassificationTestCollator

In [ ]:
# Create data collator to encode text and labels into numbers.
kpa_classification_collator = Euphemism22ClassificationTestCollator(use_tokenizer=tokenizer, 
                                                          max_sequence_len=max_length)


print('Dealing with Test...')
# Create pytorch dataset.
test_dataset = Euphemism22TestDataset(path='Euphemism22',
                                  split='test')
print('Created `test_dataset` with %d examples!'%len(test_dataset))

# Move pytorch dataset into dataloader.
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=kpa_classification_collator)
print('Created `test_dataloader` with %d batches!'%len(test_dataloader))

Dealing with Test...
Created `test_dataset` with 393 examples!
Created `test_dataloader` with 393 batches!


In [ ]:
from torch.nn.functional import softmax

In [ ]:
predictions_labels = []
predictions_probs = []
# Evaluate data for one epoch
for batch in tqdm(test_dataloader, total=len(test_dataloader)):

    # move batch to device
    batch = {k:v.type(torch.long).to(device) for k,v in batch.items() if k != 'id'}

    # Telling the model not to compute or store gradients, saving memory and
    # speeding up validation
    with torch.no_grad():        

        # Forward pass, calculate logit predictions.
        # This will return the logits rather than the loss because we have
        # not provided labels.
        # token_type_ids is the same as the "segment ids", which 
        # differentiates sentence 1 and 2 in 2-sentence tasks.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(**batch)

        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple along with the logits. We will use logits
        # later to to calculate training accuracy.        
        # Move logits and labels to CPU
        logits = outputs['logits'].detach().cpu()
        
        # get predicitons to list
        predict_content = logits.numpy().argmax(axis=-1).flatten().tolist()
        predict_prob = softmax(logits).numpy().flatten().tolist()

        # update list
        predictions_labels += predict_content        
        predictions_probs += predict_prob

  0%|          | 0/393 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
with open('answer.txt','w') as fp:
  for i, p in enumerate(predictions_labels):
    fp.write('{},{}\n'.format(i,p))

In [ ]:
import json

In [ ]:
with open('probs.json','w') as fp:
  json.dump({'probs':predictions_probs},fp)

In [ ]:
model_name_or_path = 'cardiffnlp/twitter-roberta-base-offensive'

In [ ]:
# Get model configuration.
print('Loading configuraiton...')
# model_config = XLNetConfig.from_pretrained(pretrained_model_name_or_path=model_name_or_path, num_labels=2)
model_config = AutoConfig.from_pretrained(model_name_or_path)

# Get model's tokenizer.
print('Loading tokenizer...')
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name_or_path)
# default to left padding
tokenizer.padding_side = "left"
# Define PAD Token = EOS Token = 50256
tokenizer.pad_token = tokenizer.eos_token


# Get the actual model.
print('Loading model...')
model = AutoModelForSequenceClassification.from_pretrained(pretrained_model_name_or_path=model_name_or_path)
model.classifier.out_proj = torch.nn.Linear(in_features=768, out_features=2, bias=True)
model.num_labels = 2

# resize model embedding to match new tokenizer
model.resize_token_embeddings(len(tokenizer))

# fix model padding token id
model.config.pad_token_id = model.config.eos_token_id

# Load model to defined device.
model.to(device)
print('Model loaded to `%s`'%device)

Loading configuraiton...
Loading tokenizer...
vocab_file vocab.json
merges_file merges.txt
tokenizer_file tokenizer.json
added_tokens_file added_tokens.json
special_tokens_map_file special_tokens_map.json
tokenizer_config_file tokenizer_config.json
Loading model...
Model loaded to `cuda`


In [ ]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = lr_param, # default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # default is 1e-8.
                  )

# Total number of training steps is number of batches * number of epochs.
# `train_dataloader` contains batched data so `len(train_dataloader)` gives 
# us the number of batches.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

# Store the average loss after each epoch so we can plot them.
all_loss = {'train_loss':[], 'val_loss':[]}
all_acc = {'train_acc':[], 'val_acc':[]}
all_f1 = {'train_f1':[], 'val_f1':[]}

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
# Loop through each epoch.
best_f1 = 0.0
epochs = 13
print('Epoch')
for epoch in tqdm(range(epochs)):
    print()
    print('Training {} on batches...'.format(epoch))
    # Perform one full pass over the training set.
    train_labels, train_predict, train_loss = train(train_dataloader, valid_dataloader, optimizer, scheduler, device)
    train_acc = accuracy_score(train_labels, train_predict)
    train_f1 = f1_score(train_labels, train_predict)

    # Get prediction form model on validation data. 
    print('Validation on batches...')
    valid_labels, valid_predict, val_loss = validation(valid_dataloader, device)
    val_acc = accuracy_score(valid_labels, valid_predict)
    val_f1 = f1_score(valid_labels, valid_predict)

    # Print loss and accuracy values to see how training evolves.
    print("epoch: %d  train_loss: %.5f - val_loss: %.5f - train_acc: %.5f - valid_acc: %.5f"%(epoch+1, train_loss, val_loss, train_acc, val_acc))
    print("epoch: %d  train_f1: %.5f - val_f1: %.5f -"%(epoch+1, train_f1, val_f1))
    print()

    # Store the loss value for plotting the learning curve.
    all_loss['train_loss'].append(train_loss)
    all_loss['val_loss'].append(val_loss)
    all_acc['train_acc'].append(train_acc)
    all_acc['val_acc'].append(val_acc)
    all_f1['train_f1'].append(train_f1)
    all_f1['val_f1'].append(val_f1)
    if train_f1 >= best_f1:
        model.save_pretrained("XLNet/lr_{}_best.pt".format(lr_param))
        best_f1 = train_f1
        print("model saved   best_valid_f1: %.5f"%(train_f1))

Epoch


  0%|          | 0/13 [00:00<?, ?it/s]


Training 0 on batches...


  0%|          | 0/66 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
model.eval();

In [ ]:
predictions_labels2 = []
predictions_probs2 = []
# Evaluate data for one epoch
for batch in tqdm(test_dataloader, total=len(test_dataloader)):

    # move batch to device
    batch = {k:v.type(torch.long).to(device) for k,v in batch.items() if k != 'id'}

    # Telling the model not to compute or store gradients, saving memory and
    # speeding up validation
    with torch.no_grad():        

        # Forward pass, calculate logit predictions.
        # This will return the logits rather than the loss because we have
        # not provided labels.
        # token_type_ids is the same as the "segment ids", which 
        # differentiates sentence 1 and 2 in 2-sentence tasks.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(**batch)

        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple along with the logits. We will use logits
        # later to to calculate training accuracy.        
        # Move logits and labels to CPU
        logits = outputs['logits'].detach().cpu()
        
        # get predicitons to list
        predict_content = logits.numpy().argmax(axis=-1).flatten().tolist()
        predict_prob = softmax(logits).numpy().flatten().tolist()

        # update list
        predictions_labels2 += predict_content        
        predictions_probs2 += predict_prob

In [ ]:
with open('probs2.json','w') as fp:
  json.dump({'probs':predictions_probs2},fp)

In [ ]:
with open('answer2.txt','w') as fp:
  for i, p in enumerate(predictions_labels):
    fp.write('{},{}\n'.format(i,p))

In [ ]:
predictions_probs_avg = [[(i1+j1)/2.] for i1,j1 in zip(predictions_probs,predictions_probs2)]

In [ ]:
len(predictions_probs)

786

In [ ]:
len(predictions_probs2)

786

In [ ]:
idx = 0
preds = []
while idx < len(predictions_probs_avg):
  if (predictions_probs_avg[idx] < predictions_probs_avg[idx+1]):
    preds.append(1)
  else:
    preds.append(0)
  idx += 2

In [ ]:
len(preds)

393

In [ ]:
with open('answer_avg.txt','w') as fp:
  for i, p in enumerate(preds):
    fp.write('{},{}\n'.format(i,p))